<a href="https://colab.research.google.com/github/fkonrad97/Network/blob/main/Colab/Graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [110]:
from collections import defaultdict
import pandas as pd
import numpy as np
from math import log, e
import csv

In [111]:
# Print out the 'list' to the 'name.csv' file 

def printOut(name, list):
    with open(name + '.csv', 'w', newline='') as csv_1:
        csv_out = csv.writer(csv_1)
        csv_out.writerows([list[index]] for index in range(0, len(list)))

In [112]:
# Read positions of nodes (X,Y,Z)

def positionRead(name):
    positions = pd.read_csv(name + ".csv", header=None, sep=";")
    # Remove a plus sign from the end of the number
    positions[0][0] = positions[0][0][:-1]
    positions[0] = positions[0].astype(float)    # Convert data to numerical value
    return positions

In [113]:
# Read connection table between nodes

def connectionRead(name):
    connections = pd.read_csv(name + ".csv", header=None)
    return connections

In [114]:
# List of the positions of nodes
positions = positionRead("Network/Brain_data/Brain1Positions")

# List of how nodes connected to each other
connections = connectionRead("Network/Brain_data/Brain1Connections")

C:\Users\fkonr\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [115]:
# Calculate distance between 2 points in Euclidean space
# Positions should be 3D (X,Y,Z)

def Euclidean_dist(p1, p2):
    point1 = np.array((positions[0][p1], positions[1][p1], positions[2][p1]))
    point2 = np.array((positions[0][p2], positions[1][p2], positions[2][p2]))
    return np.linalg.norm(point1 - point2)

In [116]:
# Entropy by numpy library

def entropy(labels, base=None):
    n_labels = len(labels)
    
    if n_labels <= 1:
        return 0
    
    value, counts = np.unique(labels, return_counts=True)
    probs = counts/n_labels
    n_classes = np.count_nonzero(probs)
    
    if n_classes <= 1:
        return 0
    
    ent = 0.
    
    base = e if base is None else base
    for i in probs:
        ent -= i * log(i, base)
        
    return ent

In [117]:
# Get neighbours of 'p' from the connections dataframe 

def get_neighbours(p): 
    neighbours = []
    
    for i in range(len(positions[0])):
        if connections[i][p] == 1:
            neighbours.append(i)
    
    return neighbours

In [118]:
positions

,0,1,2
0,86.828723,55.964199,26.615948
1,68.934166,51.139614,32.673099
2,99.234568,29.370370,38.259259
3,101.797312,49.265398,28.711646
4,65.881081,61.874595,43.203784
...,...,...,...
78,125.030392,96.151471,28.293137
79,114.409524,80.682993,26.676190
80,130.959165,111.471842,12.879515
81,128.398802,94.169108,12.902796


In [119]:
connections

,0,1,2,3,4,5,6,7,8,9,...,73,74,75,76,77,78,79,80,81,82
0,0,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,1
1,1,0,0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1,0,0,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,1,0,0,0,1,1,0,0,...,0,0,0,1,0,0,1,0,0,0
4,1,1,0,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,0,0,0,0,0,0,1,0,0,1,...,1,1,1,1,1,0,1,1,1,1
79,0,0,0,1,0,0,0,1,0,0,...,0,0,1,1,1,1,0,1,1,1
80,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,0,1,1
81,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,0,1


In [120]:
def toDictionary(positions, connections):
  edges = []
  for i in range(len(connections[0])):
    for j in range(len(connections[0])):
      if i!=j and i<j and connections[i][j]==1:
        edges.append((i,j,Euclidean_dist(i,j)))
  return edges

In [121]:
class Graph():
    def __init__(self):
        """
        self.edges is a dict of all possible next nodes
        e.g. {'X': ['A', 'B', 'C', 'E'], ...}
        self.weights has all the weights between two nodes,
        with the two nodes as a tuple as the key
        e.g. {('X', 'A'): 7, ('X', 'B'): 2, ...}
        self.shortest_path is a dict of node - full distance pairs
        e.g init node is 1 -> {8: (4, 60.511292298086815)} means from init node 1 through node 4 to node 8, the full distance between node 1 and node 8 is 60.51...
        self.node2node_paths is a dict of node and node to node ditances
        e.g init node is 1 -> {8: (4, 45.166701725970064)} means from node 4 to node 8 the distance is 45.166...
        """
        self.edges = defaultdict(list)
        self.weights = {}
        self.shortest_paths = {}
        self.node2node_paths = {}
#------------------------------------------------------------------------------------------------------------------------------------    
    def add_edge(self, from_node, to_node, weight):
        # Note: assumes edges are bi-directional
        self.edges[from_node].append(to_node)
        self.edges[to_node].append(from_node)
        self.weights[(from_node, to_node)] = weight
        self.weights[(to_node, from_node)] = weight
#--------------------------------------------------------------------------------------------------------------------------------------
    def dijsktra(self, initial, end):
      # shortest paths is a dict of nodes
      # whose value is a tuple of (previous node, weight)
      current_node = initial
      visited = set()
      self.shortest_paths = {initial: (None, 0)}
      self.node2node_paths = {initial: (None, 0)}
    
      while current_node != end:
        visited.add(current_node)
        destinations = graph.edges[current_node]
        weight_to_current_node = self.shortest_paths[current_node][1]

        for next_node in destinations:
            nodeDist = graph.weights[(current_node, next_node)]
            weight = graph.weights[(current_node, next_node)] + weight_to_current_node
            if next_node not in self.shortest_paths:
                self.shortest_paths[next_node] = (current_node, weight)
                self.node2node_paths[next_node] = (current_node, nodeDist)
            else:
                current_shortest_weight = self.shortest_paths[next_node][1]
                if current_shortest_weight > weight:
                    self.shortest_paths[next_node] = (current_node, weight)
                    self.node2node_paths[next_node] = (current_node, nodeDist)
        
        next_destinations = {node: self.shortest_paths[node] for node in self.shortest_paths if node not in visited}
        if not next_destinations:
            return "Route Not Possible"
        # next node is the destination with the lowest weight
        current_node = min(next_destinations, key=lambda k: next_destinations[k][1])
#-----------------------------------------------------------------------------------------------------------------------------------------------
    def BFS_SP(self, initial, end):
      explored = []
     
      # Queue for traversing the
      # graph in the BFS
      queue = [[initial]]
     
      # If the desired node is
      # reached
      if initial == end:
        print("Same Node")
        return
     
      # Loop to traverse the graph
      # with the help of the queue
      while queue:
        path = queue.pop(0)
        node = path[-1]
         
        # Condition to check if the
        # current node is not visited
        if node not in explored:
            neighbours = self.edges[node]
             
            # Loop to iterate over the
            # neighbours of the node
            for neighbour in neighbours:
                new_path = list(path)
                new_path.append(neighbour)
                queue.append(new_path)
                 
                # Condition to check if the
                # neighbour node is the goal
                if neighbour == end:
                    return new_path
            explored.append(node)
 
      # Condition when the nodes
      # are not connected
      print("Path doesn't exist.")
      return
#------------------------------------------------------------------------------------------------------------------------------------    
    def findAllBFS(self, initial):
      paths = []
      for i in range(len(connections[0])):
        if i!=initial:
          paths.append(self.BFS_SP(initial, i))
      return paths
#------------------------------------------------------------------------------------------------------------------------------------    
    def findAllDijkstra(self, initial):
      for i in range(len(connections)):
        if i!=initial:
          self.dijsktra(initial, i)
          if(len(self.shortest_paths)==len(connections[0])):
            break;
#------------------------------------------------------------------------------------------------------------------------------------    
    def shortestPath(self, initial, end):
      # Work back through destinations in shortest path
      current_node = end
      path = []
      while current_node is not None:
        path.append([current_node, self.node2node_paths[current_node][1]])
        next_node = self.shortest_paths[current_node][0]
        current_node = next_node
      # Reverse path
      path = path[::-1]
      return path

In [122]:
# Initialize graph

graph = Graph()
edges = toDictionary(positions, connections)

for edge in edges:
    graph.add_edge(*edge)

In [147]:
# Calculate all shortest paths (Dijkstra)

nodesDijk = []
distDijk = []
pathsDijk = []

for i in range(len(connections[0])):
  graph.findAllDijkstra(i)
  for j in range(len(connections[0])):
    pathsDijk.append(graph.shortestPath(i,j))

cnt = 0
for i in pathsDijk:
  path_nodes = []
  path_dist = []
  cnt += 1
  for j in range(len(i)-1):
    path_nodes.append(i[j+1][0])
    path_dist.append(i[j+1][1])
  nodesDijk.append(path_nodes)
  distDijk.append(path_dist)

nodesDijk = nodesDijk[1:]
nodesDijk[9]

[35, 10]

In [146]:
# Calculate from node '0' shortest paths (BFS)

nodesBFS = []
pathsBFS = []

for i in range(len(connections[0])):
  pathsBFS.append(graph.findAllBFS(i))

cnt = 0
for i in pathsBFS:
  for j in i:
    cnt += 1
    path_nodes = []
    for z in range(len(j)-1):
      path_nodes.append(j[z+1])
      path_dist.append(j[z+1])
    nodesBFS.append(path_nodes)

nodesBFS[9]

[7, 10]

In [151]:
for i in range(100):
  if nodesBFS[i] != nodesDijk[i]:
    print(nodesBFS[i]," - ",nodesDijk[i])

[7, 10]  -  [35, 10]
[7, 14]  -  [40, 39, 14]
[5, 16]  -  [33, 16]
[3, 19]  -  [36, 19]
[17, 20]  -  [37, 20]
[4, 21]  -  [37, 21]
[17, 25]  -  [40, 25]
[24, 26]  -  [27, 26]
[17, 32]  -  [33, 32]
[6, 39]  -  [40, 39]
[17, 41]  -  [3, 44, 41]
[82, 42]  -  [3, 44, 41, 42]
[6, 45]  -  [3, 44, 41, 45]
[4, 46]  -  [38, 76, 46]
[1, 47]  -  [3, 47]
[1, 48]  -  [3, 48]
[4, 49]  -  [35, 49]
[5, 50]  -  [38, 50]
[7, 51]  -  [38, 76, 51]
[7, 53]  -  [3, 11, 53]
[7, 54]  -  [38, 76, 54]
[17, 55]  -  [35, 75, 55]
[17, 56]  -  [38, 76, 56]
[4, 46, 57]  -  [38, 78, 57]
[17, 58]  -  [37, 58]
[82, 59]  -  [38, 78, 59]
[17, 60]  -  [34, 60]
[17, 61]  -  [37, 61]
[17, 62]  -  [38, 78, 62]
[17, 63]  -  [82, 63]
[22, 64]  -  [82, 64]
[1, 47, 65]  -  [38, 81, 65]
[1, 48, 66]  -  [38, 81, 67, 66]
[6, 39, 67]  -  [38, 81, 67]
[18, 74, 68]  -  [38, 81, 68]
[1, 47, 69]  -  [38, 78, 69]
[82, 70]  -  [38, 78, 70]
[4, 46, 71]  -  [38, 77, 74, 71]
[82, 72]  -  [38, 77, 72]
[5, 9, 73]  -  [38, 78, 73]
[18, 74]  -  

In [129]:
ent_dijk = []
ent_nodes_dijk = []

for i in distDijk:
  ent_dijk.append(entropy(i))

for i in nodesDijk:
  ent_nodes_dijk.append(entropy(i))

len(ent_dijk)

6889

In [148]:
ent_nodes_bfs = []

for i in nodesBFS:
  ent_nodes_bfs.append(entropy(i))

len(ent_nodes_bfs)

6806

In [152]:
cnt = 0
for i in range(100):
  if ent_nodes_bfs[i] != ent_nodes_dijk[i]:
    cnt += 1
    print(ent_nodes_bfs[i]," - ",ent_nodes_dijk[i])

cnt

0.6931471805599453  -  0
0.6931471805599453  -  1.0986122886681096
0  -  0.6931471805599453
0.6931471805599453  -  0
0  -  0.6931471805599453
0.6931471805599453  -  0
0  -  0.6931471805599453
0.6931471805599453  -  0
0  -  0.6931471805599453
0.6931471805599453  -  0
0  -  0.6931471805599453
0.6931471805599453  -  0
0.6931471805599453  -  1.0986122886681096
0.6931471805599453  -  1.3862943611198906
0.6931471805599453  -  1.3862943611198906
0.6931471805599453  -  1.0986122886681096
0.6931471805599453  -  1.0986122886681096
0.6931471805599453  -  1.0986122886681096
0.6931471805599453  -  1.0986122886681096
0.6931471805599453  -  1.0986122886681096
0.6931471805599453  -  1.0986122886681096
0.6931471805599453  -  1.0986122886681096
0.6931471805599453  -  1.0986122886681096
1.0986122886681096  -  0.6931471805599453
1.0986122886681096  -  1.3862943611198906
0.6931471805599453  -  1.0986122886681096
0.6931471805599453  -  1.3862943611198906
0.6931471805599453  -  1.0986122886681096
0.693147180

41

In [127]:
# printOut('Network/Data/Centroid1/entNodesBFS1', ent_nodes)